# Mount Drive and Import dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls /content/drive/My\ Drive/CSE/

biLstm_model.pkl     train.csv	      val_split.csv
glove.840B.300d.txt  train_split.csv


In [0]:
!cp /content/drive/My\ Drive/CSE/train_split.csv /content/train_split.csv
!cp /content/drive/My\ Drive/CSE/val_split.csv /content/val_split.csv
!cp /content/drive/My\ Drive/CSE/glove.840B.300d.txt /content/glove.840B.300d.txt

In [0]:
!pip install pandas==1.0.0

import numpy as pd
import pandas as pd


     |████████████████████████████████| 10.1MB 2.6MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=0.25.0; python_version >= "3.0", but you'll have pandas 1.0.0 which is incompatible.
  Found existing installation: pandas 0.25.3
    Uninstalling pandas-0.25.3:
      Successfully uninstalled pandas-0.25.3


In [0]:
# Read the raw dataset
df = pd.read_csv('train_split.csv')
df.head()

# Pytorch Model

In [0]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
import torchtext
import nltk
import time
from datetime import timedelta
import numpy as np
from sklearn import metrics


def save_model(model, model_path):
    """Save model."""
    torch.save(model.state_dict(), model_path)


def load_model(model, model_path, use_cuda=False):
    """Load model."""
    map_location = 'cpu'
    if use_cuda and torch.cuda.is_available():
        map_location = 'cuda:0'
    model.load_state_dict(torch.load(model_path, map_location))
    return model

In [0]:
use_gpu = True
device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Model, Trainer, Predictor

In [0]:
NEG_INF = -10000
TINY_FLOAT = 1e-6

def mask_softmax(matrix, mask=None):
    """Perform softmax on length dimension with masking.

    Parameters
    ----------
    matrix: torch.float, shape [batch_size, .., max_len]
    mask: torch.long, shape [batch_size, max_len]
        Mask tensor for sequence.

    Returns
    -------
    output: torch.float, shape [batch_size, .., max_len]
        Normalized output in length dimension.
    """

    if mask is None:
        result = F.softmax(matrix, dim=-1)
    else:
        mask_norm = ((1 - mask) * NEG_INF).to(matrix)
        for i in range(matrix.dim() - mask_norm.dim()):
            mask_norm = mask_norm.unsqueeze(1)
        result = F.softmax(matrix + mask_norm, dim=-1)

    return result


In [0]:
def mask_mean(seq, mask=None):
    """Compute mask average on length dimension.

    Parameters
    ----------
    seq : torch.float, size [batch, max_seq_len, n_channels],
        Sequence vector.
    mask : torch.long, size [batch, max_seq_len],
        Mask vector, with 0 for mask.

    Returns
    -------
    mask_mean : torch.float, size [batch, n_channels]
        Mask mean of sequence.
    """

    if mask is None:
        return torch.mean(seq, dim=1)

    mask_sum = torch.sum(  # [b,msl,nc]->[b,nc]
        seq * mask.unsqueeze(-1).float(), dim=1)
    seq_len = torch.sum(mask, dim=-1)  # [b]
    mask_mean = mask_sum / (seq_len.unsqueeze(-1).float() + TINY_FLOAT)

    return mask_mean

In [0]:
def mask_max(seq, mask=None):
    """Compute mask max on length dimension.

    Parameters
    ----------
    seq : torch.float, size [batch, max_seq_len, n_channels],
        Sequence vector.
    mask : torch.long, size [batch, max_seq_len],
        Mask vector, with 0 for mask.

    Returns
    -------
    mask_max : torch.float, size [batch, n_channels]
        Mask max of sequence.
    """

    if mask is None:
        return torch.mean(seq, dim=1)

    torch
    mask_max, _ = torch.max(  # [b,msl,nc]->[b,nc]
        seq + (1 - mask.unsqueeze(-1).float()) * NEG_INF,
        dim=1)

    return mask_max


In [0]:
def seq_mask(seq_len, max_len):
    """Create sequence mask.

    Parameters
    ----------
    seq_len: torch.long, shape [batch_size],
        Lengths of sequences in a batch.
    max_len: int
        The maximum sequence length in a batch.

    Returns
    -------
    mask: torch.long, shape [batch_size, max_len]
        Mask tensor for sequence.
    """

    idx = torch.arange(max_len).to(seq_len).repeat(seq_len.size(0), 1)
    mask = torch.gt(seq_len.unsqueeze(1), idx).to(seq_len)

    return mask

In [0]:
class DynamicLSTM(nn.Module):
    """
    Dynamic LSTM module, which can handle variable length input sequence.

    Parameters
    ----------
    input_size : input size
    hidden_size : hidden size
    num_layers : number of hidden layers. Default: 1
    dropout : dropout rate. Default: 0.5
    bidirectional : If True, becomes a bidirectional RNN. Default: False.

    Inputs
    ------
    input: tensor, shaped [batch, max_step, input_size]
    seq_lens: tensor, shaped [batch], sequence lengths of batch

    Outputs
    -------
    output: tensor, shaped [batch, max_step, num_directions * hidden_size],
         tensor containing the output features (h_t) from the last layer
         of the LSTM, for each t.
    """

    def __init__(self, input_size, hidden_size=100,
                 num_layers=1, dropout=0., bidirectional=False):
        super(DynamicLSTM, self).__init__()

        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, bias=True,
            batch_first=True, dropout=dropout, bidirectional=bidirectional)

    def forward(self, x, seq_lens):
        # sort input by descending length
        _, idx_sort = torch.sort(seq_lens, dim=0, descending=True)
        _, idx_unsort = torch.sort(idx_sort, dim=0)
        x_sort = torch.index_select(x, dim=0, index=idx_sort)
        seq_lens_sort = torch.index_select(seq_lens, dim=0, index=idx_sort)

        # pack input
        x_packed = pack_padded_sequence(
            x_sort, seq_lens_sort, batch_first=True)

        # pass through rnn
        y_packed, _ = self.lstm(x_packed)

        # unpack output
        y_sort, length = pad_packed_sequence(y_packed, batch_first=True)

        # unsort output to original order
        y = torch.index_select(y_sort, dim=0, index=idx_unsort)

        return y

In [0]:
class Model(nn.Module):
    """Model for quora insincere question classification.
    """

    def __init__(self, args):
        super(Model, self).__init__()

        vocab_size = args["vocab_size"]
        pretrained_embed = args["pretrained_embed"]
        padding_idx = args["padding_idx"]
        embed_dim = 300
        num_classes = 1
        num_layers = 2
        hidden_dim = 50
        dropout = 0.5

        if pretrained_embed is None:
            self.embed = nn.Embedding(vocab_size, embed_dim)
        else:
            self.embed = nn.Embedding.from_pretrained(
                pretrained_embed, freeze=False)
        self.embed.padding_idx = padding_idx

        self.rnn = DynamicLSTM(
            embed_dim, hidden_dim, num_layers=num_layers,
            dropout=dropout, bidirectional=True)

        self.fc_att = nn.Linear(hidden_dim * 2, 1)

        self.fc = nn.Linear(hidden_dim * 6, hidden_dim)
        self.act = nn.ReLU()
        self.drop = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_dim, num_classes)

        self.loss = nn.BCEWithLogitsLoss()

    def forward(self, word_seq, seq_len):
        # mask
        max_seq_len = torch.max(seq_len)
        mask = seq_mask(seq_len, max_seq_len)  # [b,msl]

        # embed
        e = self.drop(self.embed(word_seq))  # [b,msl]->[b,msl,e]

        # bi-rnn
        r = self.rnn(e, seq_len)  # [b,msl,e]->[b,msl,h*2]

        # attention
        att = self.fc_att(r).squeeze(-1)  # [b,msl,h*2]->[b,msl]
        att = mask_softmax(att, mask)  # [b,msl]
        r_att = torch.sum(att.unsqueeze(-1) * r, dim=1)  # [b,h*2]

        # pooling
        r_avg = mask_mean(r, mask)  # [b,h*2]
        r_max = mask_max(r, mask)  # [b,h*2]
        r = torch.cat([r_avg, r_max, r_att], dim=-1)  # [b,h*6]

        # feed-forward
        f = self.drop(self.act(self.fc(r)))  # [b,h*6]->[b,h]
        logits = self.out(f).squeeze(-1)  # [b,h]->[b]

        return logits


In [0]:
class Trainer(object):
    """Trainer."""

    def __init__(self, **kwargs):
        self.n_epochs = kwargs["epochs"]
        self.batch_size = kwargs["batch_size"]
        self.validate = kwargs["validate"]
        self.save_best_dev = kwargs["save_best_dev"]
        self.use_cuda = kwargs["use_cuda"]
        self.print_every_step = kwargs["print_every_step"]
        self.optimizer = kwargs["optimizer"]
        self.model_path = kwargs["model_path"]
        self.eval_metrics = kwargs["eval_metrics"]

        self._best_accuracy = 0.0

        self.device = 'cpu'
        if torch.cuda.is_available() and self.use_cuda:
            self.device = 'cuda:0'

    def train(self, network, train_data, dev_data=None):
        # transfer model to gpu if available
        network = network.to(self.device)

        # define batch iterator
        train_iter = torchtext.data.Iterator(
            dataset=train_data, batch_size=self.batch_size,
            train=True, shuffle=True, sort=False,
            device=self.device)

        # define Tester over dev data
        if self.validate:
            default_valid_args = {
                "batch_size": max(8, self.batch_size // 10),
                "use_cuda": self.use_cuda}
            validator = Tester(**default_valid_args)

        start = time.time()
        for epoch in range(1, self.n_epochs + 1):
            # turn on network training mode
            network.train()

            # initialize iterator
            train_iter.init_epoch()

            # one forward and backward pass
            self._train_step(
                train_iter, network, start=start,
                n_print=self.print_every_step, epoch=epoch)

            # validation
            if self.validate:
                if dev_data is None:
                    raise RuntimeError(
                        "self.validate is True in trainer, "
                        "but dev_data is None."
                        " Please provide the validation data.")
                eval_results = validator.test(network, dev_data)

                if self.save_best_dev and self.best_eval_result(eval_results):
                    save_model(network, self.model_path)
                    print("Saved better model selected by validation.")

    def _train_step(self, data_iterator, network, **kwargs):
        """Training process in one epoch.
        """
        step = 0
        for batch in data_iterator:
            (text, text_len), target = batch.text, batch.target

            self.optimizer.zero_grad()
            logits = network(text, text_len)
            loss = network.loss(logits, target.float())
            loss.backward()
            self.optimizer.step()

            if kwargs["n_print"] > 0 and step % kwargs["n_print"] == 0:
                end = time.time()
                diff = timedelta(seconds=round(end - kwargs["start"]))
                print_output = "[epoch: {:>3} step: {:>4}]" \
                    " train loss: {:>4.6} time: {}".format(
                        kwargs["epoch"], step, loss.item(), diff)
                print(print_output)

            step += 1

    def best_eval_result(self, eval_results):
        """Check if the current epoch yields better validation results.

        :param eval_results: dict, format {metrics_name: value}
        :return: bool, True means current results on dev set is the best.
        """
        assert self.eval_metrics in eval_results, \
            "Evaluation doesn't contain metrics '{}'." \
            .format(self.eval_metrics)

        accuracy = eval_results[self.eval_metrics]
        if accuracy > self._best_accuracy:
            self._best_accuracy = accuracy
            return True
        else:
            return False

In [0]:
class Tester(object):
    """Tester."""

    def __init__(self, **kwargs):
        self.batch_size = kwargs["batch_size"]
        self.use_cuda = kwargs["use_cuda"]
        self.device = 'cpu'
        if torch.cuda.is_available() and self.use_cuda:
            self.device = 'cuda:0'

    def test(self, network, dev_data, threshold=0.33):
        # transfer model to gpu if available
        network = network.to(self.device)

        # turn on the testing mode; clean up the history
        network.eval()
        output_list = []
        truth_list = []

        # define batch iterator
        data_iter = torchtext.data.Iterator(
            dataset=dev_data, batch_size=self.batch_size,
            train=False, device=self.device, sort=False)

        # predict
        for batch in data_iter:
            text, target = batch.text, batch.target

            with torch.no_grad():
                prediction = network(*text)

            output_list.append(prediction.detach())
            truth_list.append(target.detach())

        # evaluate
        eval_results = self.evaluate(output_list, truth_list, threshold)
        print("[tester] {}".format(self.print_eval_results(eval_results)))

        return eval_results

    def evaluate(self, predict, truth, threshold=0.33):
        """Compute evaluation metrics.

        :param predict: list of Tensor
        :param truth: list of dict
        :param threshold: threshold of positive probability
        :return eval_results: dict, format {name: metrics}.
        """
        y_trues, y_preds = [], []
        for y_true, logit in zip(truth, predict):
            y_pred = (torch.sigmoid(logit) > threshold).long().cpu().numpy()
            y_true = y_true.cpu().numpy()
            y_trues.append(y_true)
            y_preds.append(y_pred)
        y_true = np.concatenate(y_trues, axis=0)
        y_pred = np.concatenate(y_preds, axis=0)

        precision = metrics.precision_score(y_true, y_pred, pos_label=1)
        recall = metrics.recall_score(y_true, y_pred, pos_label=1)
        f1 = metrics.f1_score(y_true, y_pred, pos_label=1)

        metrics_dict = {"precision": precision, "recall": recall, "f1": f1}

        return metrics_dict

    def print_eval_results(self, results):
        """Override this method to support more print formats.
        :param results: dict, (str: float) is (metrics name: value)
        """
        return ", ".join(
            [str(key) + "=" + "{:.4f}".format(value)
             for key, value in results.items()])

In [0]:
class Predictor(object):
    """An interface for predicting outputs based on trained models.
    """

    def __init__(self, batch_size=8, use_cuda=False):
        self.batch_size = batch_size
        self.use_cuda = use_cuda

        self.device = 'cpu'
        if torch.cuda.is_available() and self.use_cuda:
            self.device = 'cuda:0'

    def predict(self, network, data, threshold=0.33):
        # transfer model to gpu if available
        network = network.to(self.device)

        # turn on the testing mode; clean up the history
        network.eval()
        output_list = []
        truth_list = []

        # define batch iterator
        data_iter = torchtext.data.Iterator(
            dataset=data, batch_size=self.batch_size,
            train=False, device=self.device, sort=False)

        for batch in data_iter:
            text, target = batch.text, batch.target

            with torch.no_grad():
                prediction = network(*text)

            output_list.append(prediction.detach())
            truth_list.append(target.detach())

        # evaluate
        eval_results = self.evaluate(output_list, truth_list, threshold)
        print("[tester] {}".format(self.print_eval_results(eval_results)))

        return eval_results

    def evaluate(self, predict, truth, threshold=0.33):
        """Compute evaluation metrics.

        :param predict: list of Tensor
        :param truth: list of dict
        :param threshold: threshold of positive probability
        :return eval_results: dict, format {name: metrics}.
        """
        y_trues, y_preds = [], []
        for y_true, logit in zip(truth, predict):
            y_pred = (torch.sigmoid(logit) > threshold).long().cpu().numpy()
            y_true = y_true.cpu().numpy()
            y_trues.append(y_true)
            y_preds.append(y_pred)
        y_true = np.concatenate(y_trues, axis=0)
        y_pred = np.concatenate(y_preds, axis=0)

        precision = metrics.precision_score(y_true, y_pred, pos_label=1)
        recall = metrics.recall_score(y_true, y_pred, pos_label=1)
        f1 = metrics.f1_score(y_true, y_pred, pos_label=1)

        metrics_dict = {"precision": precision, "recall": recall, "f1": f1}

        return metrics_dict

    def print_eval_results(self, results):
        """Override this method to support more print formats.
        :param results: dict, (str: float) is (metrics name: value)
        """
        return ", ".join(
            [str(key) + "=" + "{:.4f}".format(value)
             for key, value in results.items()])

In [0]:
train_path = './train_split.csv'
test_path = './val_split.csv'
embed_path = './glove.840B.300d.txt'
model_path = '/content/drive/My Drive/CSE/biLstm_model.pkl'

In [0]:
def pre():
    """Pre-process model."""

    print("Pre-processing...")

    # load data
    fix_length = 100
    text = torchtext.data.Field(
        sequential=True, use_vocab=True, lower=True,
        tokenize=nltk.word_tokenize, batch_first=True,
        is_target=False, fix_length=fix_length,
        include_lengths=True)
    target = torchtext.data.Field(
        sequential=False, use_vocab=False,
        batch_first=True, is_target=True)
    dataset = torchtext.data.TabularDataset(
        train_path, format='csv',
        fields={"cleaned_text": ('text', text),
                "target": ('target', target)})
    data_test = torchtext.data.TabularDataset(
        test_path, format='csv',
        fields={"cleaned_text": ('text', text),
                "target": ('target', target)})

    # build vocab
    text.build_vocab(dataset, data_test, min_freq=3)
    text.vocab.load_vectors(torchtext.vocab.Vectors(embed_path))
    vocab_size = len(text.vocab.itos)
    padding_idx = text.vocab.stoi[text.pad_token]

    # split data
    data_train, data_val = dataset.split(split_ratio=0.9)

    print("train set size:", len(data_train))
    print("val set size:", len(data_val))
    print("test set size:", len(data_test))
    print("vocab size:", len(text.vocab.itos))
    print("embed shape:", text.vocab.vectors.shape)
    print('')

    args_dict = {
        "data_train": data_train, "data_val": data_val,
        "data_test": data_test, "vocab_size": vocab_size,
        "padding_idx": padding_idx}

    return args_dict

In [0]:
import nltk
nltk.download('punkt')
args = pre()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Pre-processing...
train set size: 881632
val set size: 97959
test set size: 326531
vocab size: 68947
embed shape: torch.Size([68947, 300])



In [0]:
def train(**args):
    """Train model.
    """

    print("Training...")

    # load data and embed
    data_train = args["data_train"]
    pretrained_embed = data_train.fields["text"].vocab.vectors

    # define model
    model_args = {
        "vocab_size": args["vocab_size"],
        "padding_idx": args["padding_idx"],
        "pretrained_embed": pretrained_embed,
    }
    model = Model(model_args)
    load_model(model, model_path, use_cuda=True)

    # define trainer
    trainer_args = {
        "epochs": 50,
        "batch_size": 128,
        "validate": True,
        "save_best_dev": True,
        "use_cuda": True,
        "print_every_step": 1000,
        "optimizer": torch.optim.Adam(model.parameters(), lr=1e-4),
        "model_path": model_path,
        "eval_metrics": "f1",
    }
    trainer = Trainer(**trainer_args)

    # train
    data_val = args["data_val"]
    trainer.train(model, data_train, dev_data=data_val)

    print('')

In [0]:
train(**args)

In [0]:
def test(**args):
    """Train model.
    """

    print("Testing...")

    # define model
    model_args = {
        "vocab_size": args["vocab_size"],
        "padding_idx": args["padding_idx"],
        "pretrained_embed": None,
    }
    model = Model(model_args)
    load_model(model, model_path, use_cuda=True)

    # define tester
    tester_args = {
        "batch_size": 128,
        "use_cuda": True,
    }
    tester = Tester(**tester_args)

    # test and threshold selection
    data_val = args["data_val"]
    best_thresh, best_f1 = 0., 0.
    for thresh in np.arange(0.1, 0.501, 0.01):
        thresh = np.round(thresh, 2)
        f1 = tester.test(model, data_val, threshold=thresh)["f1"]
        print("threshold: {:>.2f} f1: {}".format(thresh, f1))
        if f1 > best_f1:
            best_thresh, best_f1 = thresh, f1

    args["threshold"] = best_thresh

    print("best f1 on dev: {} threshold: {:>.2f}".format(best_f1, best_thresh))
    print('')

    return args

In [0]:
 args = test(**args)

Testing...
[tester] precision=0.4983, recall=0.8465, f1=0.6273
threshold: 0.10 f1: 0.6273220049942141
[tester] precision=0.5100, recall=0.8374, f1=0.6339
threshold: 0.11 f1: 0.6339430135622746
[tester] precision=0.5193, recall=0.8286, f1=0.6385
threshold: 0.12 f1: 0.6384649483883225
[tester] precision=0.5286, recall=0.8202, f1=0.6429
threshold: 0.13 f1: 0.6428755475392941
[tester] precision=0.5370, recall=0.8133, f1=0.6468
threshold: 0.14 f1: 0.6468396627230538
[tester] precision=0.5457, recall=0.8074, f1=0.6512
threshold: 0.15 f1: 0.6511997878828053
[tester] precision=0.5535, recall=0.8000, f1=0.6543
threshold: 0.16 f1: 0.6542985817033004
[tester] precision=0.5605, recall=0.7937, f1=0.6571
threshold: 0.17 f1: 0.6570515001020478
[tester] precision=0.5695, recall=0.7867, f1=0.6607
threshold: 0.18 f1: 0.6606847045831034
[tester] precision=0.5752, recall=0.7788, f1=0.6617
threshold: 0.19 f1: 0.6616856364778996
[tester] precision=0.5809, recall=0.7715, f1=0.6628
threshold: 0.20 f1: 0.66280

# Testing Model on validation set

In [0]:
args["threshold"] = 0.28

In [0]:
def infer(**args):
    """Inference using model.
    """

    print("Predicting...")

    # define model
    model_args = {
        "vocab_size": args["vocab_size"],
        "padding_idx": args["padding_idx"],
        "pretrained_embed": None,
    }
    model = Model(model_args)
    load_model(model, model_path, use_cuda=True)

    # define predictor
    predictor = Predictor(batch_size=128, use_cuda=False)

    # predict
    data_test = args["data_test"]
    threshold = args["threshold"]
    y_pred = predictor.predict(model, data_test, threshold=threshold)

   

In [0]:
infer(**args)

Predicting...
[tester] precision=0.6235, recall=0.7313, f1=0.6731
